In [2]:
import os
import timeit

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix)
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
import os
base_dir = "/Users/adii/Downloads/archive"
train_path = os.path.join(base_dir, 'train')
valid_path = os.path.join(base_dir, 'val')
test_path = os.path.join(base_dir, 'test')


In [4]:
model_name = "LeNet-5"
class_names = ['Healthy', 'Doubtful', 'Minimal', 'Moderate', 'Severe']

target_size = (224, 224)
epochs = 100
batch_size = 256
img_shape = (224, 224, 1)

Augmentation and Normalization

In [5]:
aug_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input,
    horizontal_flip=True,
    brightness_range=[0.3, 0.8],
    width_shift_range=[-50, 0, 50, 30, -30],
    zoom_range=0.1,
    fill_mode="nearest",
)

noaug_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input,
)

In [6]:
train_generator = aug_datagen.flow_from_directory(
    train_path, class_mode="categorical", target_size=target_size, color_mode="grayscale", shuffle=True
)

test_generator = noaug_datagen.flow_from_directory(
    test_path, class_mode="categorical", target_size=target_size, color_mode="grayscale", shuffle=False,
)

valid_generator = noaug_datagen.flow_from_directory(
    valid_path, class_mode="categorical", target_size=target_size, color_mode="grayscale", shuffle=False,
)


Found 5778 images belonging to 5 classes.
Found 1656 images belonging to 5 classes.
Found 826 images belonging to 5 classes.


In [7]:
y_train = train_generator.labels
y_val = valid_generator.labels
y_test = test_generator.labels

In [8]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(224, 224, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 6)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 44944)             0         
                                                                 
 dense (Dense)               (None, 120)               5

In [9]:
history = model.fit(train_generator, validation_data=valid_generator, epochs = 50)

Epoch 1/50


2023-10-29 10:39:13.008186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-10-29 10:39:13.032241: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


181/181 [==============================] - ETA: 0s - loss: 1.4295 - accuracy: 0.3903

2023-10-29 10:39:48.709193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


181/181 [==============================] - 37s 204ms/step - loss: 1.4295 - accuracy: 0.3903 - val_loss: 1.6247 - val_accuracy: 0.3995
Epoch 2/50
181/181 [==============================] - 36s 198ms/step - loss: 1.4062 - accuracy: 0.3958 - val_loss: 1.4497 - val_accuracy: 0.3971
Epoch 3/50
181/181 [==============================] - 35s 193ms/step - loss: 1.4039 - accuracy: 0.3932 - val_loss: 1.4248 - val_accuracy: 0.3571
Epoch 4/50
181/181 [==============================] - 36s 199ms/step - loss: 1.4015 - accuracy: 0.3967 - val_loss: 1.4453 - val_accuracy: 0.3959
Epoch 5/50
181/181 [==============================] - 37s 202ms/step - loss: 1.3976 - accuracy: 0.3994 - val_loss: 1.4370 - val_accuracy: 0.3317
Epoch 6/50
181/181 [==============================] - 35s 195ms/step - loss: 1.3968 - accuracy: 0.3979 - val_loss: 1.4275 - val_accuracy: 0.3777
Epoch 7/50
181/181 [==============================] - 37s 203ms/step - loss: 1.3958 - accuracy: 0.3969 - val_loss: 1.4629 - val_accuracy: 0.3

In [10]:
test_generator = noaug_datagen.flow_from_directory(
    test_path,
    class_mode="categorical",
    target_size=target_size,
    color_mode="grayscale",
    shuffle=False,
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 1656 images belonging to 5 classes.
 1/52 [..............................] - ETA: 7s - loss: 0.8144 - accuracy: 0.6562

2023-10-29 11:10:37.840773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


52/52 [==============================] - 6s 113ms/step - loss: 1.1577 - accuracy: 0.5326
Test Loss: 1.1577379703521729
Test Accuracy: 0.532608687877655
